In [1]:
#packrat::init("/work/postresearch/Shared/Projects/Farbod/CaseMix")
#install.packages("remotes")
#remotes::install_github("ellessenne/comorbidity@0.5.3")

setwd("/work/postresearch/Shared/Projects/Farbod")
#packrat::on()
options(repr.matrix.max.rows=100, repr.matrix.max.cols=300)
options(repr.plot.width = 20, repr.plot.height = 15)

numcores=50
numcores_foreach=50

library(tidyverse)
library(parallel)
library(data.table)
library(fst)
library(comorbidity)
library(zeallot)
library(reshape)
library(dtplyr)
library(haven)
library(vroom)
library(dplyr)
`%!in%` = Negate(`%in%`)

setDTthreads(numcores)


library(foreach)
library(doMC)
registerDoMC(cores=numcores_foreach)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.0     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose



Attaching package: ‘reshape’


The following object is masked from ‘package:data.table’:

    melt


The following object is masked from ‘package:dplyr’:

    rename


The following objects are masked from ‘package:tidyr’:

    expand, smiths



Attaching package: 

In [2]:
carrier_data_all_years = read_fst(
    "carrier_data_all_years.fst", as.data.table = T)

In [3]:
yearly_calculations <-
    vroom("Integration_Intensity/yearly_calculations_with_new_charlson_year_of_diagnosis.csv",
        num_threads = numcores
    ) %>% as.data.table()
yearly_calculations[, 1:=NULL, with = F]

New names:
• `` -> `...1`
Rows: 14554 Columns: 704
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (227): ENTITLEMENT_BUY_IN_IND01_2013, ENTITLEMENT_BUY_IN_IND02_2013, EN...
dbl  (445): ...1, DESY_SORT_KEY, tot_allowed_carrier, office_visit_count, of...
lgl   (25): stable_angina, unstable_angina, MI, cardiac_arrest, stroke, hype...
date   (7): first_diagnosis, MI_date, catheterization_date, angioplasty_date...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message in `[.data.table`(yearly_calculations, , `:=`(1, NULL), with = F):
“with=FALSE ignored, it isn't needed when using :=. See ?':=' for examples.”


In [6]:
yearly_calculations[, first_diagnosis := as.IDate(first_diagnosis)]

In [7]:
head(carrier_data_all_years)
head(yearly_calculations)

DESY_SORT_KEY,CLAIM_NO,LINE_NUM,CLM_THRU_DT,LINE_PLACE_OF_SRVC_CD,HCPCS_CD,LINE_ICD_DGNS_VRSN_CD,LINE_ICD_DGNS_CD,LINE_ALOWD_CHRG_AMT,PRF_PHYSN_NPI,PRVDR_SPCLTY,PRVDR_STATE_CD,date,year,month_year
<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<dbl>,<chr>,<chr>,<int>,<IDate>,<dbl>,<chr>
100000015,2,1,20130425,22,94375,9,496,15.26,1073503884,29,22,2013-04-25,2013,2013-04
100000015,2,2,20130425,22,94726,9,496,13.54,1073503884,29,22,2013-04-25,2013,2013-04
100000015,2,3,20130425,22,94729,9,496,9.95,1073503884,29,22,2013-04-25,2013,2013-04
100000015,3,1,20130528,11,99214,9,41400,114.64,1285600932,11,22,2013-05-28,2013,2013-05
100000015,3,2,20130528,11,93000,9,41400,20.08,1285600932,11,22,2013-05-28,2013,2013-05
100000015,4,1,20130719,22,99213,9,496,51.76,1659344091,29,22,2013-07-19,2013,2013-07


DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,MI_date,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HM

In [8]:
office_visit_codes=c("99201","99202","99203","99204","99205","99211","99212","99213","99214"
                     ,"99215")

# Most common physicians for each patient

Now, we will find most common physicians and cardiologists for each patient.


In [32]:
# adding most common physicians
add_patient_NPI_last_year = function(data, summary_data, time_frame = 365) {
  comorbidity_and_phys_data =
    inner_join(data, summary_data[, c("DESY_SORT_KEY", "first_diagnosis")],
      by = "DESY_SORT_KEY"
    ) %>%
    filter(date - first_diagnosis < 0 &
      first_diagnosis - date <= time_frame) %>%
    as.data.table()

  patient_NPI_count_finder = function(data) {
    result = data %>%
      mutate(is_office_visit = HCPCS_CD %in% office_visit_codes) %>%
      group_by(DESY_SORT_KEY, PRF_PHYSN_NPI) %>%
      summarise(n = sum(is_office_visit, na.rm = T)) %>%
      filter(n > 0) %>%
      arrange(.by_group = T, desc(n))
  }

  patient_NPI_counts = patient_NPI_count_finder(comorbidity_and_phys_data)

  patient_NPI_counts = left_join(patient_NPI_counts,
    distinct(data[, .(PRF_PHYSN_NPI, PRVDR_SPCLTY)]),
    by = "PRF_PHYSN_NPI"
  )

  find_most_common = function(data) {
    data %>%
      group_by(DESY_SORT_KEY) %>%
      arrange(.by_group = T, desc(n)) %>%
      slice(1) %>%
      as.data.table()
  }

  find_most_common_by_specialty = function(data, specialty_code) {
    data %>%
      filter(PRVDR_SPCLTY %in% specialty_code) %>%
      group_by(DESY_SORT_KEY) %>%
      arrange(.by_group = T, desc(n)) %>%
      slice(1) %>%
      as.data.table()
  }

  most_common_physician = find_most_common(patient_NPI_counts)
  # primary care = 01:general practice/ family practice:08/ internal medicine:11/ geriatrics:38
  most_common_primary_care_physician = find_most_common_by_specialty(patient_NPI_counts,
    specialty_code = c("01", "08", "11", "38")
  )
  most_common_cardiologists = find_most_common_by_specialty(patient_NPI_counts, specialty_code = c("06", "C3"))
  # most_common_interventional_cardiologists = find_most_common_by_specialty(patient_NPI_counts, specialty_code = "C3")

  most_common_physician = data.frame(most_common_physician) %>%
    rename_with(~ paste0("most_common_physician_last_year_", .x))
  most_common_primary_care_physician = data.frame(most_common_primary_care_physician) %>%
    rename_with(~ paste0("most_common_primary_care_physician_last_year_", .x))
  most_common_cardiologists = data.frame(most_common_cardiologists) %>%
    rename_with(~ paste0("most_common_cardiologist_last_year_", .x))
  # most_common_interventional_cardiologists = data.frame(most_common_interventional_cardiologists) %>%
  # rename_with( ~ paste0("most_common_interventional_cardiologist_", .x))

  summary_data = left_join(
    summary_data,
    most_common_physician,
    by = c("DESY_SORT_KEY" = "most_common_physician_last_year_DESY_SORT_KEY")
  )
  summary_data = left_join(
    summary_data,
    most_common_primary_care_physician,
    by = c("DESY_SORT_KEY" = "most_common_primary_care_physician_last_year_DESY_SORT_KEY")
  )
  summary_data = left_join(
    summary_data,
    most_common_cardiologists,
    by = c("DESY_SORT_KEY" = "most_common_cardiologist_last_year_DESY_SORT_KEY")
  ) %>%
    as.data.table()

  # summary_data = left_join(
  #  summary_data,
  #  most_common_interventional_cardiologists,
  #  by = c("DESY_SORT_KEY" = "most_common_interventional_cardiologist_DESY_SORT_KEY")
  # )%>%
  #  as.data.table()

  # summary_data[, year_first_diagnosis := lubridate::year(first_diagnosis)]%>%
  # as.data.table()
}

yearly_calculations_with_npi <-
  add_patient_NPI_last_year(data = carrier_data_all_years, summary_data = yearly_calculations)
head(yearly_calculations_with_npi)

`summarise()` has grouped output by 'DESY_SORT_KEY'. You can override using the `.groups` argument.


DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,MI_date,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HM

In [12]:
physician_integration_stats <- read_fst(
    "results_apr/physician_integration_stats.fst",
    as.data.table = T
)

In [14]:
head(physician_integration_stats)

PRF_PHYSN_NPI,month_year,in_facility_visits_count,in_all_visits_count,in_facility_non_exclusive_HCPCS_count,in_all_non_exclusive_HCPCS_count,in_facility_count,in_all_count,date,in_facility_visits_prp,in_facility_non_exclusive_HCPCS_prp,in_facility_prp
<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<IDate>,<dbl>,<dbl>,<dbl>
,2013-01,0,1,0,2,0,3,2013-01-01,0,0,0
,2013-02,0,6,0,16,0,20,2013-02-01,0,0,0
,2013-03,0,3,0,6,0,6,2013-03-01,0,0,0
,2013-04,0,5,0,7,0,17,2013-04-01,0,0,0
,2013-05,0,3,0,5,0,8,2013-05-01,0,0,0
,2013-06,0,2,0,5,0,15,2013-06-01,0,0,0


In [18]:
# rename columns
rename_last <- function(data, how_many, new_names) {
  total_cols <- ncol(data)
  setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
}
add_integration_status_last_year <- function(data, physician_integration_stats) {
  data <- data[, c(
    "DESY_SORT_KEY",
    "first_diagnosis",
    "most_common_physician_last_year_PRF_PHYSN_NPI",
    "most_common_primary_care_physician_last_year_PRF_PHYSN_NPI",
    "most_common_cardiologist_last_year_PRF_PHYSN_NPI"
    # ,"most_common_interventional_cardiologist_PRF_PHYSN_NPI"
  )]

  most_common_physician <- left_join(
    data,
    physician_integration_stats,
    by = c(
      "most_common_physician_last_year_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI"
    )
  ) %>% as.data.table()
  rename_last(
    most_common_physician,
    ncol(physician_integration_stats) - 1,
    paste("most_common_physician_last_year_", colnames(physician_integration_stats)[2:ncol(physician_integration_stats)], sep = "")
  )
  most_common_physician <- most_common_physician[, -c(
    "most_common_primary_care_physician_last_year_PRF_PHYSN_NPI",
    "most_common_cardiologist_last_year_PRF_PHYSN_NPI"
    # ,"most_common_interventional_cardiologist_PRF_PHYSN_NPI"
  )]


  most_common_primary_care <- left_join(
    data,
    physician_integration_stats,
    by = c(
      "most_common_primary_care_physician_last_year_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI"
    )
  ) %>% as.data.table()
  rename_last(
    most_common_primary_care,
    ncol(physician_integration_stats) - 1,
    paste("most_common_primary_care_physician_last_year_", colnames(physician_integration_stats)[2:ncol(physician_integration_stats)], sep = "")
  )
  most_common_primary_care <- most_common_primary_care[, -c(
    "most_common_physician_last_year_PRF_PHYSN_NPI",
    "most_common_cardiologist_last_year_PRF_PHYSN_NPI"
    # ,"most_common_interventional_cardiologist_PRF_PHYSN_NPI"
  )]

  most_common_cardiologist <- left_join(
    data,
    physician_integration_stats,
    by = c(
      "most_common_cardiologist_last_year_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI"
    )
  ) %>% as.data.table()
  rename_last(
    most_common_cardiologist,
    ncol(physician_integration_stats) - 1,
    paste("most_common_cardiologist_last_year_", colnames(physician_integration_stats)[2:ncol(physician_integration_stats)], sep = "")
  )
  most_common_cardiologist <- most_common_cardiologist[, -c(
    "most_common_physician_last_year_PRF_PHYSN_NPI",
    "most_common_primary_care_physician_last_year_PRF_PHYSN_NPI"
    # ,"most_common_interventional_cardiologist_PRF_PHYSN_NPI"
  )]

  # most_common_interventional_cardiologist = left_join(
  #  data,
  #  physician_integration_stats,
  #  by = c(
  #    "most_common_interventional_cardiologist_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI")
  # ) %>% as.data.table()
  # rename_last(
  #  most_common_interventional_cardiologist,
  #  ncol(physician_integration_stats)-1,
  #  paste("most_common_interventional_cardiologist_",colnames(physician_integration_stats)[2:ncol(physician_integration_stats)],sep="")
  #  )
  # most_common_interventional_cardiologist=most_common_interventional_cardiologist[,-c("most_common_physician_PRF_PHYSN_NPI"
  #                                                ,"most_common_primary_care_physician_PRF_PHYSN_NPI"
  #                                                ,"most_common_cardiologist_PRF_PHYSN_NPI")]

  return(list(
    most_common_physician,
    most_common_primary_care,
    most_common_cardiologist
    # most_common_interventional_cardiologist
  ))
}

melted_physician_integration_stats_last_year_stable_angina <- add_integration_status_last_year(
  data = yearly_calculations_with_npi,
  physician_integration_stats = physician_integration_stats
)

In [19]:
lapply(melted_physician_integration_stats_last_year_stable_angina, head)

DESY_SORT_KEY,first_diagnosis,most_common_physician_last_year_PRF_PHYSN_NPI,most_common_physician_last_year_month_year,most_common_physician_last_year_in_facility_visits_count,most_common_physician_last_year_in_all_visits_count,most_common_physician_last_year_in_facility_non_exclusive_HCPCS_count,most_common_physician_last_year_in_all_non_exclusive_HCPCS_count,most_common_physician_last_year_in_facility_count,most_common_physician_last_year_in_all_count,most_common_physician_last_year_date,most_common_physician_last_year_in_facility_visits_prp,most_common_physician_last_year_in_facility_non_exclusive_HCPCS_prp,most_common_physician_last_year_in_facility_prp
<int>,<IDate>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<IDate>,<dbl>,<dbl>,<dbl>
100004643,2018-07-24,1144223207,2013-01,0,0,0,0,0,0,2013-01-01,NaN,NaN,NaN
100004643,2018-07-24,1144223207,2013-09,0,0,0,0,0,0,2013-09-01,NaN,NaN,NaN
100004643,2018-07-24,1144223207,2013-12,0,0,0,0,0,0,2013-12-01,NaN,NaN,NaN
100004643,2018-07-24,1144223207,2014-12,0,0,0,0,0,0,2014-12-01,NaN,NaN,NaN
100004643,2018-07-24,1144223207,2015-01,0,0,0,0,0,0,2015-01-01,NaN,NaN,NaN
100004643,2018-07-24,1144223207,2015-02,0,0,0,0,0,0,2015-02-01,NaN,NaN,NaN
DESY_SORT_KEY,first_diagnosis,most_common_primary_care_physician_last_year_PRF_PHYSN_NPI,most_common_primary_care_physician_last_year_month_year,most_common_primary_care_physician_last_year_in_facility_visits_count,most_common_primary_care_physician_last_year_in_all_visits_count,most_common_primary_care_physician_last_year_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_last_year_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_last_year_in_facility_count,most_common_primary_care_physician_last_year_in_all_count,most_common_primary_care_physician_last_year_date,most_common_primary_care_physician_last_year_in_facility_visits_prp,most_common_primary_care_physician_last_year_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_last_year_in_facility_prp
<int>,<IDate>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<IDate>,<dbl>,<dbl>,<dbl>
100004643,2018-07-24,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


# Add integration stats to the data

#### In all codes

In [33]:
change_stats_summarizer= function(data,melted_physician_integration_stats,specialty){

  require(dtplyr)
  require(lubridate)
  require(tidyverse)
  
  looking_for=paste(specialty,"_last_year_PRF_PHYSN_NPI",sep="")
  date_looking_for=paste(specialty,"_last_year_date",sep="")
  in_facility_prp_looking_for=paste(specialty,"_last_year_in_facility_prp",sep="")
  in_all_count_looking_for=paste(specialty,"_last_year_in_all_count",sep="")
  in_facility_count_looking_for=paste(specialty,"_last_year_in_facility_count",sep="")
  
  
  result=melted_physician_integration_stats%>%
  group_by(DESY_SORT_KEY)%>%
  mutate(n_integration_stat_year_of_diagnosis=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=365 &
           !!rlang::sym(date_looking_for)-first_diagnosis>=0 &
           !is.na(!!rlang::sym(in_facility_prp_looking_for)) ,
           length(!!rlang::sym(in_facility_prp_looking_for))],
         
         n_integration_stat_2013=
         .[!!rlang::sym(date_looking_for)<as.IDate("2013-12-31") &
           !is.na(!!rlang::sym(in_facility_prp_looking_for)) ,
           length(!!rlang::sym(in_facility_prp_looking_for))],
         
         n_integration_stat_year_before_diagnosis=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=0 &
           first_diagnosis-!!rlang::sym(date_looking_for)<=365 &
           !is.na(!!rlang::sym(in_facility_prp_looking_for)) ,
           length(!!rlang::sym(in_facility_prp_looking_for))]
         
        )%>%
  
  summarise(prp_in_year_of_diagnosis_05=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=365 &
           !!rlang::sym(date_looking_for)-first_diagnosis>=0,
           sum(!!rlang::sym(in_facility_prp_looking_for)>=0.5 , na.rm=T)]/
         n_integration_stat_year_of_diagnosis,
         
         prp_in_2013_05=
         .[!!rlang::sym(date_looking_for)<as.IDate("2013-12-31"),
           sum(!!rlang::sym(in_facility_prp_looking_for)>=0.5, na.rm=T)]/
         n_integration_stat_2013,
         
         prp_in_year_before_diagnosis_05=
            .[!!rlang::sym(date_looking_for)-first_diagnosis<=0 &
              first_diagnosis-!!rlang::sym(date_looking_for)<=365,
              sum(!!rlang::sym(in_facility_prp_looking_for)>=0.5, na.rm=T)]/
         n_integration_stat_year_before_diagnosis,
         
         prp_in_year_of_diagnosis_03=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=365 &
           !!rlang::sym(date_looking_for)-first_diagnosis>=0,
           sum(!!rlang::sym(in_facility_prp_looking_for)>=0.3, na.rm=T)]/
         n_integration_stat_year_of_diagnosis,
         
         prp_in_2013_03=
         .[!!rlang::sym(date_looking_for)<as.IDate("2013-12-31"),
           sum(!!rlang::sym(in_facility_prp_looking_for)>=0.3, na.rm=T)]/
         n_integration_stat_2013,
         
         prp_in_year_before_diagnosis_03=
            .[!!rlang::sym(date_looking_for)-first_diagnosis<=0 &
              first_diagnosis-!!rlang::sym(date_looking_for)<=365,
              sum(!!rlang::sym(in_facility_prp_looking_for)>=0.3, na.rm=T)]/
         n_integration_stat_year_before_diagnosis,
         
        avg_in_year_of_diagnosis=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=365 &
           !!rlang::sym(date_looking_for)-first_diagnosis>=0,
           sum(!!rlang::sym(in_facility_count_looking_for), na.rm=T) / sum(!!rlang::sym(in_all_count_looking_for), na.rm=T)],

         
         avg_in_2013=
         .[!!rlang::sym(date_looking_for)<as.IDate("2013-12-31"),
           sum(!!rlang::sym(in_facility_count_looking_for), na.rm=T) / sum(!!rlang::sym(in_all_count_looking_for), na.rm=T)],
         
         avg_in_year_before_diagnosis=
            .[!!rlang::sym(date_looking_for)-first_diagnosis<=0 &
              first_diagnosis-!!rlang::sym(date_looking_for)<=365,
              sum(!!rlang::sym(in_facility_count_looking_for), na.rm=T) / sum(!!rlang::sym(in_all_count_looking_for), na.rm=T)]

        )%>%
  distinct()%>%
  as.data.table()
  
  #rename last n columns in a dataset
  rename_last = function(data, how_many, new_names) {
    total_cols = ncol(data)
    setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
  }
  
  rename_last(
    result,
    9,
    paste(specialty, "last_year", colnames(result)[(ncol(result) - 8):ncol(result)], sep = "_")
    )
  
  data = left_join(
    data,
    result,
    by = "DESY_SORT_KEY"
  ) %>% as.data.table()
  
  return(data)
  
}

#### Non-exclusive HCPCS codes  

In [34]:
change_stats_summarizer_non_exclusive_only = function(data,melted_physician_integration_stats,specialty){

  require(dtplyr)
  require(lubridate)
  require(tidyverse)

  looking_for=paste(specialty,"_last_year_PRF_PHYSN_NPI",sep="")
  date_looking_for=paste(specialty,"_last_year_date",sep="")
  in_facility_prp_looking_for=paste(specialty,"_last_year_in_facility_non_exclusive_HCPCS_prp",sep="")
  in_all_count_looking_for=paste(specialty,"_last_year_in_all_non_exclusive_HCPCS_count",sep="")
  in_facility_count_looking_for=paste(specialty,"_last_year_in_facility_non_exclusive_HCPCS_count",sep="")
  
  
  result=melted_physician_integration_stats%>%
  group_by(DESY_SORT_KEY)%>%
  mutate(n_integration_stat_year_of_diagnosis=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=365 &
           !!rlang::sym(date_looking_for)-first_diagnosis>=0 &
           !is.na(!!rlang::sym(in_facility_prp_looking_for)) ,
           length(!!rlang::sym(in_facility_prp_looking_for))],
         
         n_integration_stat_2013=
         .[!!rlang::sym(date_looking_for)<as.IDate("2013-12-31") &
           !is.na(!!rlang::sym(in_facility_prp_looking_for)) ,
           length(!!rlang::sym(in_facility_prp_looking_for))],
         
         n_integration_stat_year_before_diagnosis=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=0 &
           first_diagnosis-!!rlang::sym(date_looking_for)<=365 &
           !is.na(!!rlang::sym(in_facility_prp_looking_for)) ,
           length(!!rlang::sym(in_facility_prp_looking_for))]
         
        )%>%
  
  summarise(prp_in_year_of_diagnosis_05_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=365 &
           !!rlang::sym(date_looking_for)-first_diagnosis>=0,
           sum(!!rlang::sym(in_facility_prp_looking_for)>=0.5 , na.rm=T)]/
         n_integration_stat_year_of_diagnosis,
         
         prp_in_2013_05_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)<as.IDate("2013-12-31"),
           sum(!!rlang::sym(in_facility_prp_looking_for)>=0.5, na.rm=T)]/
         n_integration_stat_2013,
         
         prp_in_year_before_diagnosis_05_non_exclusive_HCPCS=
            .[!!rlang::sym(date_looking_for)-first_diagnosis<=0 &
              first_diagnosis-!!rlang::sym(date_looking_for)<=365,
              sum(!!rlang::sym(in_facility_prp_looking_for)>=0.5, na.rm=T)]/
         n_integration_stat_year_before_diagnosis,
         
         prp_in_year_of_diagnosis_03_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=365 &
           !!rlang::sym(date_looking_for)-first_diagnosis>=0,
           sum(!!rlang::sym(in_facility_prp_looking_for)>=0.3, na.rm=T)]/
         n_integration_stat_year_of_diagnosis,
         
         prp_in_2013_03_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)<as.IDate("2013-12-31"),
           sum(!!rlang::sym(in_facility_prp_looking_for)>=0.3, na.rm=T)]/
         n_integration_stat_2013,
         
         prp_in_year_before_diagnosis_03_non_exclusive_HCPCS=
            .[!!rlang::sym(date_looking_for)-first_diagnosis<=0 &
              first_diagnosis-!!rlang::sym(date_looking_for)<=365,
              sum(!!rlang::sym(in_facility_prp_looking_for)>=0.3, na.rm=T)]/
         n_integration_stat_year_before_diagnosis,
         
        avg_in_year_of_diagnosis_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=365 &
           !!rlang::sym(date_looking_for)-first_diagnosis>=0,
           sum(!!rlang::sym(in_facility_count_looking_for), na.rm=T) / sum(!!rlang::sym(in_all_count_looking_for), na.rm=T)],

         
         avg_in_2013_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)<as.IDate("2013-12-31"),
           sum(!!rlang::sym(in_facility_count_looking_for), na.rm=T) / sum(!!rlang::sym(in_all_count_looking_for), na.rm=T)],
         
         avg_in_year_before_diagnosis_non_exclusive_HCPCS=
            .[!!rlang::sym(date_looking_for)-first_diagnosis<=0 &
              first_diagnosis-!!rlang::sym(date_looking_for)<=365,
              sum(!!rlang::sym(in_facility_count_looking_for), na.rm=T) / sum(!!rlang::sym(in_all_count_looking_for), na.rm=T)]

        )%>%
  distinct()%>%
  as.data.table()
  
  #rename last n columns in a dataset
  rename_last = function(data, how_many, new_names) {
    total_cols = ncol(data)
    setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
  }
  
  rename_last(
    result,
    9,
    paste(specialty,"last_year",colnames(result)[(ncol(result)-8):ncol(result)],sep="_")
    )
  
  data = left_join(
    data,
    result,
    by = "DESY_SORT_KEY"
  ) %>% as.data.table()
  
  return(data)
  
}

#### Adding integration stats to data

In [35]:
yearly_calculations_with_npi = change_stats_summarizer(
  yearly_calculations_with_npi,
  melted_physician_integration_stats_last_year_stable_angina[[1]], "most_common_physician"
)
yearly_calculations_with_npi = change_stats_summarizer(
  yearly_calculations_with_npi,
  melted_physician_integration_stats_last_year_stable_angina[[2]], "most_common_primary_care_physician"
)
yearly_calculations_with_npi = change_stats_summarizer(
  yearly_calculations_with_npi,
  melted_physician_integration_stats_last_year_stable_angina[[3]], "most_common_cardiologist"
)

In [36]:
yearly_calculations_with_npi = change_stats_summarizer_non_exclusive_only(
  yearly_calculations_with_npi,
  melted_physician_integration_stats_last_year_stable_angina[[1]], "most_common_physician"
)
yearly_calculations_with_npi = change_stats_summarizer_non_exclusive_only(
  yearly_calculations_with_npi,
  melted_physician_integration_stats_last_year_stable_angina[[2]], "most_common_primary_care_physician"
)
yearly_calculations_with_npi = change_stats_summarizer_non_exclusive_only(
  yearly_calculations_with_npi,
  melted_physician_integration_stats_last_year_stable_angina[[3]], "most_common_cardiologist"
)

In [38]:
head(yearly_calculations_with_npi)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,MI_date,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HM

In [40]:
head(yearly_calculations_with_npi[, .(
    DESY_SORT_KEY, most_common_primary_care_physician_last_year_avg_in_year_before_diagnosis_non_exclusive_HCPCS
)], 20)

DESY_SORT_KEY,most_common_primary_care_physician_last_year_avg_in_year_before_diagnosis_non_exclusive_HCPCS
<int>,<dbl>
100004643,NaN
100008543,0.0000
100010653,0.0000
100010781,0.0000
100013703,0.0000
100013815,0.0000
100015949,0.0000
100016989,NaN
100023001,NaN


In [43]:
write.csv(yearly_calculations_with_npi, "Integration_Intensity/yearly_calculations_with_last_year_pcp.csv")

In [5]:
yearly_calculations_with_npi <- read.csv("Integration_Intensity/yearly_calculations_with_last_year_pcp.csv") %>%
    as.data.table()

In [3]:
colnames(yearly_calculations_with_npi)

[1] "X"                                                                                               
  [2] "DESY_SORT_KEY"                                                                                   
  [3] "first_diagnosis"                                                                                 
  [4] "tot_allowed_carrier"                                                                             
  [5] "office_visit_count"                                                                              
  [6] "office_visit_cost_carrier"                                                                       
  [7] "cardiology_visit_count"                                                                          
  [8] "distinct_clinicians"                                                                             
  [9] "distinct_cardiologists"                                                                          
 [10] "distinct_primary_care_physicians"                                                                
 [11] "distinct_surgeons"                                                                               
 [12] "distinct_other_specialties"                                                                      
 [13] "catheterization_count"                                                                           
 [14] "catheterization_cost_carrier"                                                                    
 [15] "ecg_count"                                                                                       
 [16] "ecg_cost_carrier"                                                                                
 [17] "cardiac_ct_count"                                                                                
 [18] "cardiac_ct_cost_carrier"                                                                         
 [19] "cardiac_mri_count"                                                                               
 [20] "cardiac_mri_cost_carrier"                                                                        
 [21] "stress_test_count"                                                                               
 [22] "stress_test_cost_carrier"                                                                        
 [23] "echocardiography_count"                                                                          
 [24] "echocardiography_cost_carrier"                                                                   
 [25] "angioplasty_count"                                                                               
 [26] "angioplasty_cost_carrier"                                                                        
 [27] "CABG_count"                                                                                      
 [28] "CABG_cost_carrier"                                                                               
 [29] "stable_angina"                                                                                   
 [30] "unstable_angina"                                                                                 
 [31] "MI"                                                                                              
 [32] "MI_date"                                                                                         
 [33] "cardiac_arrest"                                                                                  
 [34] "stroke"                                                                                          
 [35] "hypertension"                                                                                    
 [36] "copd"                                                                                            
 [37] "obesity"                                                                                         
 [38] "depression"                                                                                      
 [39] "diabe

In [8]:
data_for_brady <- data.table(
    yearly_calculations_with_npi[, .(
        desy_sort_key = DESY_SORT_KEY,
        pcp_prior_npi = most_common_primary_care_physician_last_year_PRF_PHYSN_NPI,
        pcp_prior_hopd_yrprior = most_common_primary_care_physician_last_year_avg_in_year_before_diagnosis_non_exclusive_HCPCS,
        pcp_prior_hopd_yrdx = most_common_primary_care_physician_last_year_avg_in_year_of_diagnosis_non_exclusive_HCPCS
    ) ]
)

head(data_for_brady,20)



desy_sort_key,pcp_prior_npi,pcp_prior_hopd_yrprior,pcp_prior_hopd_yrdx
<int>,<dbl>,<dbl>,<dbl>
100004643,NA,NA,NA
100008543,1033107701,0.0000,0.0000
100010653,1518020502,0.0000,0.0000
100010781,1467470047,0.0000,0.0000
100013703,1548465404,0.0000,0.0000
100013815,1235147182,0.0000,0.0000
100015949,1992907539,0.0000,0.0000
100016989,NA,NA,NA
100023001,NA,NA,NA


In [9]:
write.csv(data_for_brady, "Integration_Intensity/data_for_brady_last_year_pcp.csv")